# Goal: receive sensor data from the application PhonePi

### STARTING

See https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html

See https://github.com/jupyter/jupyter/wiki/A-gallery-of-interesting-Jupyter-Notebooks

See https://www.w3schools.com/python

The application PhonePi See: https://play.google.com/store/apps/details?id=com.phonepi

# HANDS ON  

* title
    * a
    * b
     * c

pip3 install flask
pip3 install flask_sockets
pip install paho-mqtt

RUN
python PhonePiPython3.py

$e^{i\pi} + 1 = 0$

### Install on Raspberry
    sudo route add default gw 192.168.1.1 wlan0
    ping 8.8.8.8
    python --version    :  Python 2.7.13
    sudo apt-get update
    sudo apt-get install python-smbus
    sudo pip install paho-mqtt   :  paho-mqtt-1.4.0
    
    python myMPU6050.py

# Zen of python

In [ ]:
import this

In [ ]:
import numpy             as np
import matplotlib.pyplot as plt
costs = [1.0,4.5,9.5,16.0]
#plt.ylabel('some numbers')
#plt.plot(costs)
plt.plot([1.0, 2, 3, 4.0], costs)
plt.show()

## Sender

In [ ]:
import time
import paho.mqtt.client as paho
broker="192.168.1.6"
client= paho.Client("mqttpythonsender") #create client object client1.on_publish = on_publish #assign function to callback client1.connect(broker,port) #establish connection client1.publish("house/bulb1","on")
print("connecting to broker ",broker)
client.connect(broker)                       #connect
time.sleep(1)
print("publishing ")
client.publish("unibo/livestream","2")         #publish
time.sleep(1)
print("publishing ")
client.publish("unibo/livestream","4")         #publish
time.sleep(1)
print("publishing ")
client.publish("unibo/livestream","9")         #publish
time.sleep(1)
client.disconnect()                        #disconnect
print("bye" )

## mqttReceiver.py

In [ ]:
import time
import paho.mqtt.client as paho
import matplotlib.pyplot as plt

brokerAddr="localhost"
duration = 20
data     = []
counter  = 0
firstVal = 0
rot      = 0

def on_message(client, userdata, message) :   #define callback
    global counter, rot, data, firstVal   
    if rot > 90  : 
        client.disconnect()             #disconnect
        return    
    msg      = str( message.payload.decode("utf-8")  )
    msgitems = msg.split(",")
    value    = msgitems[5]
    v       =  float( msgitems[5] )  
    #print( v )
    if counter <= 2 :
        firstVal  = v
        rot       = 0
        print("first",firstVal, "rot", rot, "count",counter)
    rot  = abs( v -  firstVal )
    print("rot:", rot, "count",counter )
    if rot < 90  :
        counter = counter + 1
        data.append( rot )  
        if rot > 85  :
        	print("WARNING ... rot=", rot )
    else :
        data.append( rot ) 
        print("first",firstVal, "rot", rot, "count",counter)
        plt.plot(data)
        plt.show()   
    
client= paho.Client("receiver")      
client.on_message=on_message            # Bind function to callback
client.connect(brokerAddr)              #connect
print("connected to broker ", brokerAddr)
print("subscribing to unibo/qak/events")
client.subscribe("unibo/qak/events")      #subscribe
client.loop_start()             #start loop to process received messages
time.sleep(duration)
client.disconnect()             #disconnect
print("bye")
client.loop_stop()              #stop loop 

   

## mqttPlotQakEvents.py

In [ ]:
import time
import paho.mqtt.client as paho
import matplotlib.pyplot as plt

brokerAddr="localhost"
duration = 10
x        = []
y        = []
z        = []
goon     = True
msgnum   = 0 
def on_message(client, userdata, message) :   #define callback
	global msgnum,  x,y,z, goon
	if not goon :
		return
	#msg(androidSensor,event,android,none,androidSensor(TYPE,X,Y,Z),MSGNUM)
    #msg(g521,event, gyroSender, none, g521(TYPE,X,Y,Z),MSGNUM)
	evMsg = str( message.payload.decode("utf-8")  )
	msgitems = evMsg.split(",")
	if msgnum < 200  :
		#print("evMsg=", evMsg, "msgnum=", msgnum )
		msgnum = msgnum + 1
		x.append( float( msgitems[5] ) )
		y.append( float( msgitems[6] ) )
		vz = float( msgitems[7].split(')')[0] )
		z.append( vz )
	else  :
		print("evMsg=", evMsg, "msgnum=", msgnum )
		plt.plot(list(x), color='red')
		#plt.plot(list(y), color='green')
		#plt.plot(list(z), color='blue')
		plt.show()
		goon = False
		client.disconnect()
    
client= paho.Client("receiver")      
client.on_message=on_message            # Bind function to callback

client.connect(brokerAddr)              #connect
print("connected to broker ", brokerAddr)
print("subscribing to unibo/qak/events")
client.subscribe("unibo/qak/events")      #subscribe

print("collecting values; please wait ..." )
client.loop_start()             #start loop to process received messages
time.sleep(duration)
client.disconnect()             #disconnect
print("bye")
client.loop_stop()              #stop loop        


### gyro.py

In [ ]:
import time
import paho.mqtt.client as paho
import matplotlib.pyplot as plt

brokerAddr="localhost"
duration  = 30
x         = []
y         = []
z         = []
r         = []
counter   = 0 
goon      = True
dt        = 0
startTime = 0
angle     = 0.0

def on_message(client, userdata, message) :   #define callback
	global counter,  goon, x, y, z, r, startTime, angle
	now     = time.time() 
	elapsed = now-startTime
	#print( "now=" , now, "elapsed=",  elapsed)
	startTime = now
	counter = counter + 1
	#msg(androidSensor,event,android,none,androidSensor(TYPE,X,Y,Z),MSGNUM)
	#msg(g521,event, gyroSender, none, g521(TYPE,X,Y,Z),MSGNUM)
	evMsg = str( message.payload.decode("utf-8")  )
	#print("evMsg=", evMsg )
	msgitems = evMsg.split(",")
	x.append( float( msgitems[5] ) )
	y.append( float( msgitems[6] ) )
	vz = float( msgitems[7].split(')')[0] )
	z.append( vz )
	
	da    = abs(vz) * elapsed
	angle = angle + da
	r.append( angle )
	print("vz=", ("%.3f" % vz), "angle=", ("%.3f" % angle), "counter=", counter)
	#print("elapsed=", ("%.2f" % elapsed), "da=", ("%.2f" % da) ) 	
	if angle > 90.0 and goon :
		plt.plot(x, color='red')
		plt.plot(y, color='green')
		plt.plot(z, color='blue')
		plt.plot(r, color='black')
		plt.show()
		goon = False
		client.disconnect()
     
plt.style.use("classic")
client= paho.Client("receiver")      
client.on_message=on_message            # Bind function to callback
client.connect(brokerAddr)              #connect
print("connected to broker ", brokerAddr)
print("subscribing to unibo/qak/events")
client.subscribe("unibo/qak/events")      #subscribe
startTime     = time.time() 
#print( "startTime=" , ti
print( "startTime=" , startTime )
client.loop_start()             #start loop to process received messages
time.sleep(duration)
client.disconnect()             #disconnect
print("bye")
client.loop_stop()              #stop loop        
     


In [ ]:
"""
rotSender.py
"""
import paho.mqtt.client as paho
import time

brokerAddr ="localhost"

def sendMsg( client,CMD ) :
	#msg(rotationCmd,event,gyroSender,none,rotationCmd(CMD),MSGNUM)	  
	msgout   = "msg(rotationCmd,event,source,none,rotationCmd("+ CMD +"))"
	print("SEND: " + msgout )
	client.publish("unibo/qak/events", msgout, 0, retain=False);

client= paho.Client("utilGY521")      
client.connect(brokerAddr)              #connect
print("connected to broker ", brokerAddr)

time.sleep(1)
sendMsg( client, 'l') 
time.sleep(2)
sendMsg( client, 'r')
time.sleep(2)
sendMsg( client, 'l')
time.sleep(2)
sendMsg( client, 'r')
print( "bye " )

In [ ]:
"""
mqttPlotQakEvents.py
"""
import time
import paho.mqtt.client as paho
import matplotlib.pyplot as plt

slotSize  = 40
msgnum    = 0
msgInSlot = 0
brokerAddr="localhost"
duration = 20
x        = []
y        = []
z        = []
goon     = True
 
def on_message(client, userdata, message) :   #define callback
	global msgnum,  x,y,z, msgInSlot, slotSize
	msgnum    = msgnum + 1
	#msg(androidSensor,event,android,none,androidSensor(TYPE,X,Y,Z),MSGNUM)
    #msg(g521,event, gyroSender, none, g521(TYPE,X,Y,Z),MSGNUM)
	evMsg = str( message.payload.decode("utf-8")  )
	msgitems = evMsg.split(",")
	if msgInSlot < slotSize  :
		msgInSlot = msgInSlot + 1
		#print( evMsg, ":", msgInSlot )
		x.append( float( msgitems[5] ) )
		y.append( float( msgitems[6] ) )
		vz = float( msgitems[7].split(')')[0] )
		z.append( vz )
	else  :
		msgInSlot = 0
		#print("evMsg=", evMsg, "msgnum=", msgnum )
		plt.plot(list(x), color='red')
		plt.plot(list(y), color='green')
		plt.plot(list(z), color='blue')
		plt.show()
    
client= paho.Client("receiver")      
client.on_message=on_message            # Bind function to callback

client.connect(brokerAddr)              #connect
print("connected to broker ", brokerAddr)
print("subscribing to unibo/qak/events")
client.subscribe("unibo/qak/events")      #subscribe

print("collecting values; please wait ..." )
client.loop_start()             #start loop to process received messages
time.sleep(duration)
client.disconnect()             #disconnect
print("bye")
client.loop_stop()              #stop loop        


In [ ]:
"""
gyro90.py
"""
import time
import paho.mqtt.client as paho

brokerAddr="localhost"
goon      = True
startTime = 0
dt        = 5
angle     = 0.0
anglePrec = 0.0
def sendDoneMsg( client  ) :
	msgout = "msg( gyro90done,event,gyro90,none,gyro90done,1 )"
	client.publish("unibo/qak/events", msgout, 0, retain=False)


def on_message(client, userdata, message) :   #define callback
	global goon,  startTime, angle, anglePrec
	if not goon :
		return
	now       = time.time() 
	elapsed   = now-startTime
	startTime = now
	#msg(androidSensor,event,android,none,androidSensor(TYPE,X,Y,Z),MSGNUM)
	#msg(g521,event, gyroSender, none, g521(TYPE,X,Y,Z),MSGNUM)
	evMsg = str( message.payload.decode("utf-8")  )
	#print("evMsg=", evMsg )
	msgitems = evMsg.split(",")
	vz    = float( msgitems[7].split(')')[0] )
	da    = abs(vz) * elapsed
	angle = angle + da
	delta = abs( angle - anglePrec )
	print("vz=", ("%.3f" % vz), "angle=", ("%.3f" % angle), "elapsed=",  elapsed, " delta=", delta   )
	#if( delta > 0.5 ) :
	#	print("vz=", ("%.3f" % vz), "angle=", ("%.3f" % angle), "elapsed=",  elapsed, " delta=", delta   )
	anglePrec = angle
	if angle > 90.0 and goon :
		goon = False
		#sendDoneMsg( client )
		if angle > 91 :
			print( "TOO MUCH !!!")
     
client= paho.Client("receiver")      
client.on_message=on_message            # Bind function to callback
client.connect(brokerAddr)              #connect
print("connected to broker ", brokerAddr)
print("subscribing to unibo/qak/events")
client.subscribe("unibo/qak/events")      #subscribe
startTime = time.time() 

client.loop_start()             #start loop to process received messages
while goon :
	time.sleep(dt)
client.disconnect()             #disconnect
print("bye")
client.loop_stop()              #stop loop      


In [ ]:
"""
rotSender.py
"""
import paho.mqtt.client as paho
import time

brokerAddr ="localhost"

def sendMsg( client,CMD ) :
	#msg(rotationCmd,event,gyroSender,none,rotationCmd(CMD),MSGNUM)	  
	msgout   = "msg(rotationCmd,event,source,none,rotationCmd("+ CMD +"))"
	print("SEND: " + msgout )
	client.publish("unibo/qak/events", msgout, 0, retain=False);

client= paho.Client("utilGY521")      
client.connect(brokerAddr)              #connect
print("connected to broker ", brokerAddr)

time.sleep(1)
sendMsg( client, 'r')
time.sleep(2)
sendMsg( client, 'z')
time.sleep(2)
sendMsg( client, 'z')
time.sleep(2)
sendMsg( client, 'z')
""" 
time.sleep(1)
sendMsg( client, 'l')
time.sleep(1)
sendMsg( client, 'z')
time.sleep(1)

sendMsg( client, 'x')
"""
print( "bye " )